<a href="https://colab.research.google.com/github/abdulkham1d0v/hackathon-projects/blob/main/%22CASE2_Airkickers_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OperaBuild Case2

## About Project


There is a set of data: a construction plan with a plan of materials for the successful completion of work, there is also data with the actual volume of materials for a certain period of time.

`Main Objective`: Develop a predictive algorithm that will predict **the lack of materials or its surplus** to amount of  a particular job.

## Project plan

1.  **Download and prepare the data.**

2. **Train and test the various models:**

    2.1. Getting features and target from prepared dataset
    
    2.2. Splitting features and target into train and test
    
    2.3.Training and testing various Classifier algorithms
    
    2.4. Saving the model

3. **General Conclusion**

### Downloading  and preparing the data. 

**WARNING**

On the cell which have located at the bottom of this cell we will install `catboost` library for further usings. If you have this library, there is no need to run this cell or you ignore it with commenting

In [ ]:
#pip install catboost

In [ ]:
#Libraries which needed to work with dataset
from json import load
from datetime import datetime


import numpy as np

#Used for data visualization as table
import pandas as pd

In [ ]:
#Predicting models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

import catboost as cb

In [ ]:
#Different tools needed to work with model
#Splitter(train ,test)
from sklearn.model_selection import train_test_split

#For saving model
import joblib

In [ ]:
ds_case_2 = [] #Here we will store our dataset

In [ ]:
#Here we have opened our dataset and saved into ds_1
with open("wbs.task2.json", 'r') as file:
    ds_2 = load(file)

#By using loop we have fetched each row one by one and extracted new and existing information
for item in ds_2:
    if item['end_date'] != None and item.get('duration') != None and item['planMaterial'] != None:
        ds_case_2.append({
            'planMaterial': item.get('planMaterial'), 
            'factMaterial': item.get('factMaterial'), 
            'start_date': item.get('start_date'), 
            'end_date': item.get('end_date'),
            'duration': item.get('duration')
        })

In [ ]:
display(pd.DataFrame(ds_case_2).head())
display(pd.DataFrame(ds_case_2).shape)

,planMaterial,factMaterial,start_date,end_date,duration
0,1.500000,15.000000,2020-11-11T10:49:23,2020-11-29T18:00:00,90
1,5.294528,52.945278,2020-12-18T10:49:41,2020-11-29T18:00:00,90
2,960.000000,100.000000,2020-11-12T03:27:10,2020-11-29T18:00:00,90
3,9.012000,90.120000,2020-12-25T03:03:54,2020-11-29T18:00:00,90
4,662.250500,575.870000,2020-09-29T09:04:56,2020-11-29T18:00:00,60


(1215, 5)

### Conclusion:

Our first step has finished. Here we have *downloaded, opened  and prepared* data for further usings. As you see above we got pretty enough data. 

## Train and test the various models:

### Getting `features` and `target` from *prepared dataset*

In [ ]:
X = []
y = []

for item in ds_case_2:
    try:
        X.append([
            float(item['factMaterial']),
            float(item['duration'])
        ])
        if item.get('planMaterial') < item.get('factMaterial'):
          y.append(0)
        elif item.get('planMaterial') > item.get('factMaterial'):
          y.append(1)
        else:
          y.append(2)
    except:
        print(item)

In [ ]:
display(len(X))
display(len(y))

1215

1215

Here we have saved our `features` into **X** list and `target` into **y**. 

In [ ]:
display(pd.DataFrame(y).value_counts())

0    648
1    487
2     80
dtype: int64

### Splitting `features` and `target` into *train* and *test*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print('X_train size:',len(X_train))
print('X_test size:',len(X_test))
print('y_train size:',len(y_train))
print('y_test size:',len(y_test))

X_train size: 911
X_test size: 304
y_train size: 911
y_test size: 304


Here we have **splitted** our `features` and `target` into test set and train set.

### Training and testing various Classifier algorithms

#### For scoring all models we will **ACCURACY_SCORE**. 

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

gnb.score(X_test, y_test)

0.5526315789473685

We have started from **GaussianNB**. And we got not bad results. At least our model works better than `random` model **xD**

In [ ]:
clf = LogisticRegression(random_state=12345, class_weight='balanced')
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.40131578947368424

Then, we have `trained` and `tested` **Logistic Classifier model**. This model did `quite worse than previous one`. Let's see is there any other models that can do better than *these models*? )

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

0.6875

Wow, **KNeighbors** did this  better than previous ones. Is there any other effective ones than **KNeighbors**? Hmmm....

In [ ]:
for depth in range(1,51,1):
    model = DecisionTreeClassifier(random_state = 12345,max_depth = depth,class_weight='balanced' )
    model.fit(X_train, y_train)
    print('{} estimator score: {}'.format(depth, model.score(X_test, y_test)))

1 estimator score: 0.40789473684210525
2 estimator score: 0.34210526315789475
3 estimator score: 0.5230263157894737
4 estimator score: 0.5394736842105263
5 estimator score: 0.5592105263157895
6 estimator score: 0.631578947368421
7 estimator score: 0.625
8 estimator score: 0.618421052631579
9 estimator score: 0.6677631578947368
10 estimator score: 0.6381578947368421
11 estimator score: 0.6644736842105263
12 estimator score: 0.7006578947368421
13 estimator score: 0.6973684210526315
14 estimator score: 0.7039473684210527
15 estimator score: 0.7039473684210527
16 estimator score: 0.7072368421052632
17 estimator score: 0.7039473684210527
18 estimator score: 0.7039473684210527
19 estimator score: 0.7039473684210527
20 estimator score: 0.7039473684210527
21 estimator score: 0.7039473684210527
22 estimator score: 0.7039473684210527
23 estimator score: 0.7039473684210527
24 estimator score: 0.7039473684210527
25 estimator score: 0.7039473684210527
26 estimator score: 0.7039473684210527
27 estim

Here we have tested **DecisionTreeClassifier** by using loop. We got best about `70.7`  ~ `71` 

In [ ]:
for estimator in range(10,301,10):
    model = RandomForestClassifier(random_state = 12345,n_estimators = estimator,class_weight='balanced')
    model.fit(X_train, y_train)
    print('{} estimator score: {}'.format(estimator, model.score(X_test, y_test)))

10 estimator score: 0.6973684210526315
20 estimator score: 0.7203947368421053
30 estimator score: 0.7171052631578947
40 estimator score: 0.7072368421052632
50 estimator score: 0.7072368421052632
60 estimator score: 0.7072368421052632
70 estimator score: 0.7072368421052632
80 estimator score: 0.7072368421052632
90 estimator score: 0.7072368421052632
100 estimator score: 0.7072368421052632
110 estimator score: 0.7072368421052632
120 estimator score: 0.7072368421052632
130 estimator score: 0.7072368421052632
140 estimator score: 0.7072368421052632
150 estimator score: 0.7072368421052632
160 estimator score: 0.7072368421052632
170 estimator score: 0.7072368421052632
180 estimator score: 0.7072368421052632
190 estimator score: 0.7072368421052632
200 estimator score: 0.7105263157894737
210 estimator score: 0.7072368421052632
220 estimator score: 0.7105263157894737
230 estimator score: 0.7072368421052632
240 estimator score: 0.7105263157894737
250 estimator score: 0.7105263157894737
260 estim

Now we have tested **RandomForestClassifier** by using `for` loop. We got best  **72%** 

Now we are going to use **CatBoost** `gradient booster` from **Yandex**. This algorithm very similar to the Neural Networks it will learn it's own mistake.

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

model = cb.CatBoostClassifier()
grid = {'iterations': [100,250, 300, 400],
        'learning_rate': [0.06,0.2],
        'depth': [13, 18],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]
       }
model.grid_search(grid, train_dataset)

Выходные данные были обрезаны до нескольких последних строк (5000).
166:	learn: 0.2509856	test: 0.6177137	best: 0.5747228 (59)	total: 15.1s	remaining: 12s
167:	learn: 0.2505572	test: 0.6179979	best: 0.5747228 (59)	total: 15.2s	remaining: 11.9s
168:	learn: 0.2501290	test: 0.6183023	best: 0.5747228 (59)	total: 15.3s	remaining: 11.9s
169:	learn: 0.2498037	test: 0.6179613	best: 0.5747228 (59)	total: 15.4s	remaining: 11.8s
170:	learn: 0.2495925	test: 0.6201399	best: 0.5747228 (59)	total: 15.5s	remaining: 11.7s
171:	learn: 0.2491804	test: 0.6189261	best: 0.5747228 (59)	total: 15.6s	remaining: 11.6s
172:	learn: 0.2488472	test: 0.6187687	best: 0.5747228 (59)	total: 15.7s	remaining: 11.5s
173:	learn: 0.2485069	test: 0.6198862	best: 0.5747228 (59)	total: 15.8s	remaining: 11.5s
174:	learn: 0.2479738	test: 0.6201958	best: 0.5747228 (59)	total: 15.9s	remaining: 11.4s
175:	learn: 0.2477945	test: 0.6212008	best: 0.5747228 (59)	total: 16s	remaining: 11.3s
176:	learn: 0.2473295	test: 0.6210700	best: 0.

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

In [ ]:
model.is_fitted()

True

Now our model is *fitted*. And let's **test** our model `in action!`

In [ ]:
print("Testing performance")
print(model.score(X_test, y_test))

Testing performance
0.694078947368421


In this case **CatBoosterClassifier** was an weaker or have we tuned it's hyperparameters wrongly? )



In [ ]:
final_model = RandomForestClassifier(random_state=12345, class_weight='balanced', n_estimators = 20)
final_model.fit(X_train, y_train)
print('Accuracy score for RFC algorithm about: {:.2f}'.format(final_model.score(X_test,y_test)))

Accuracy score for RFC algorithm about: 0.72


### Saving the model

#### Use cell on the below for loading your model.

In [ ]:
#path = 'Path for saving your file\name_for_file.joblib'
#joblib.dump(final_model, 'final_case_2_72.joblib')

## General Conclusion

We have downloaded and prepared data for making predictions. Then we have divided `features` and `target` from data. After that we splitted `features` and `target` into `train` and `test`. Then we have performed prediction with various **Classification models**. According results which we get **RandomForestClassifier**  was the `best`. **KNeighborsClassifier**  about **~3 - 4%**  was `less accurate` than **previous**, **LogisticRegression** was `less accurate`  more than  **1.5 times** than the **best**. **CatBooster algorithm** `slowest one`. 